In [107]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, String, MetaData,Text,BigInteger,Integer,Float
import pandas as pd
import psycopg2
import json 

config_file=open('config.json')
config=json.load(config_file)

param_dic=config['pg_details']

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)

engine = create_engine(connect,echo=False)
## Data Extraction From Postgres SQL
df = pd.read_sql_table('dem', engine)
df

,Date,Net Asset Value,Repurchase Price,Sale Price
0,30-08-2016,11.1403,11.1124,11.1403
1,31-08-2016,11.1454,11.1175,11.1454
2,01-09-2016,11.1413,11.1134,11.1413
3,02-09-2016,11.1301,11.1023,11.1301
4,06-09-2016,11.1481,11.1202,11.1481
...,...,...,...,...
253,19-09-2017,10.8483,10.8212,10.8483
254,20-09-2017,10.8645,10.8373,10.8645
255,21-09-2017,10.8048,10.7778,10.8048
256,22-09-2017,10.8073,10.7803,10.8073


In [108]:
df.head()

,Date,Net Asset Value,Repurchase Price,Sale Price
0,30-08-2016,11.1403,11.1124,11.1403
1,31-08-2016,11.1454,11.1175,11.1454
2,01-09-2016,11.1413,11.1134,11.1413
3,02-09-2016,11.1301,11.1023,11.1301
4,06-09-2016,11.1481,11.1202,11.1481


# Transform

## Checking For Missing Values in DataFrame

In [109]:
df.isna().sum()

Date                 0
Net Asset Value     15
Repurchase Price    11
Sale Price          12
dtype: int64

In [110]:
df.shape

(258, 4)

In [111]:
df.dtypes

Date                 object
Net Asset Value     float64
Repurchase Price    float64
Sale Price          float64
dtype: object

In [112]:
df['Net Asset Value_1']= df['Net Asset Value'].interpolate()
df

,Date,Net Asset Value,Repurchase Price,Sale Price,Net Asset Value_1
0,30-08-2016,11.1403,11.1124,11.1403,11.1403
1,31-08-2016,11.1454,11.1175,11.1454,11.1454
2,01-09-2016,11.1413,11.1134,11.1413,11.1413
3,02-09-2016,11.1301,11.1023,11.1301,11.1301
4,06-09-2016,11.1481,11.1202,11.1481,11.1481
...,...,...,...,...,...
253,19-09-2017,10.8483,10.8212,10.8483,10.8483
254,20-09-2017,10.8645,10.8373,10.8645,10.8645
255,21-09-2017,10.8048,10.7778,10.8048,10.8048
256,22-09-2017,10.8073,10.7803,10.8073,10.8073


In [113]:
df = df.interpolate()
df

,Date,Net Asset Value,Repurchase Price,Sale Price,Net Asset Value_1
0,30-08-2016,11.1403,11.1124,11.1403,11.1403
1,31-08-2016,11.1454,11.1175,11.1454,11.1454
2,01-09-2016,11.1413,11.1134,11.1413,11.1413
3,02-09-2016,11.1301,11.1023,11.1301,11.1301
4,06-09-2016,11.1481,11.1202,11.1481,11.1481
...,...,...,...,...,...
253,19-09-2017,10.8483,10.8212,10.8483,10.8483
254,20-09-2017,10.8645,10.8373,10.8645,10.8645
255,21-09-2017,10.8048,10.7778,10.8048,10.8048
256,22-09-2017,10.8073,10.7803,10.8073,10.8073


In [114]:
df[ 'saleMV' ] = df['Sale Price'].rolling(2).mean()

In [115]:
df['saleStd'] = df['Sale Price'].rolling(2).std()

In [116]:
df

,Date,Net Asset Value,Repurchase Price,Sale Price,Net Asset Value_1,saleMV,saleStd
0,30-08-2016,11.1403,11.1124,11.1403,11.1403,NaN,NaN
1,31-08-2016,11.1454,11.1175,11.1454,11.1454,11.14285,0.003606
2,01-09-2016,11.1413,11.1134,11.1413,11.1413,11.14335,0.002899
3,02-09-2016,11.1301,11.1023,11.1301,11.1301,11.13570,0.007920
4,06-09-2016,11.1481,11.1202,11.1481,11.1481,11.13910,0.012728
...,...,...,...,...,...,...,...
253,19-09-2017,10.8483,10.8212,10.8483,10.8483,10.84725,0.001485
254,20-09-2017,10.8645,10.8373,10.8645,10.8645,10.85640,0.011455
255,21-09-2017,10.8048,10.7778,10.8048,10.8048,10.83465,0.042214
256,22-09-2017,10.8073,10.7803,10.8073,10.8073,10.80605,0.001768


In [117]:
# df.set_index('Date')

In [118]:
df['Date']=pd.to_datetime(df['Date'])

In [119]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

In [120]:
df

,Date,Net Asset Value,Repurchase Price,Sale Price,Net Asset Value_1,saleMV,saleStd,year,month
0,2016-08-30,11.1403,11.1124,11.1403,11.1403,NaN,NaN,2016,8
1,2016-08-31,11.1454,11.1175,11.1454,11.1454,11.14285,0.003606,2016,8
2,2016-01-09,11.1413,11.1134,11.1413,11.1413,11.14335,0.002899,2016,1
3,2016-02-09,11.1301,11.1023,11.1301,11.1301,11.13570,0.007920,2016,2
4,2016-06-09,11.1481,11.1202,11.1481,11.1481,11.13910,0.012728,2016,6
...,...,...,...,...,...,...,...,...,...
253,2017-09-19,10.8483,10.8212,10.8483,10.8483,10.84725,0.001485,2017,9
254,2017-09-20,10.8645,10.8373,10.8645,10.8645,10.85640,0.011455,2017,9
255,2017-09-21,10.8048,10.7778,10.8048,10.8048,10.83465,0.042214,2017,9
256,2017-09-22,10.8073,10.7803,10.8073,10.8073,10.80605,0.001768,2017,9


# Load

In [121]:
from pymongo import MongoClient

In [122]:
conn=config["mongo_details"]['host']+":"+str(config["mongo_details"]['port'])+"/"

In [123]:
conn

'mongodb://localhost:27017/'

In [128]:
myclient = MongoClient(conn)
mydb = myclient["etl"]
mycol = mydb["etl"]

In [129]:
# df=df.reset_index(inplace=True) # Reset Index
data_dict = df.to_dict("records")

In [130]:
data_dict

[{'Date': Timestamp('2016-08-30 00:00:00'),
  'Net Asset Value': 11.1403,
  'Repurchase Price': 11.1124,
  'Sale Price': 11.1403,
  'Net Asset Value_1': 11.1403,
  'saleMV': nan,
  'saleStd': nan,
  'year': 2016,
  'month': 8},
 {'Date': Timestamp('2016-08-31 00:00:00'),
  'Net Asset Value': 11.1454,
  'Repurchase Price': 11.1175,
  'Sale Price': 11.1454,
  'Net Asset Value_1': 11.1454,
  'saleMV': 11.14285,
  'saleStd': 0.0036062445840517802,
  'year': 2016,
  'month': 8},
 {'Date': Timestamp('2016-01-09 00:00:00'),
  'Net Asset Value': 11.1413,
  'Repurchase Price': 11.1134,
  'Sale Price': 11.1413,
  'Net Asset Value_1': 11.1413,
  'saleMV': 11.14335,
  'saleStd': 0.0028991378028663905,
  'year': 2016,
  'month': 1},
 {'Date': Timestamp('2016-02-09 00:00:00'),
  'Net Asset Value': 11.1301,
  'Repurchase Price': 11.1023,
  'Sale Price': 11.1301,
  'Net Asset Value_1': 11.1301,
  'saleMV': 11.1357,
  'saleStd': 0.007919595949287024,
  'year': 2016,
  'month': 2},
 {'Date': Timestamp('

In [131]:
mycol.insert_many(data_dict)